In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from dataloader import get_dataloaders
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformer import Transformer,TransformerEncoder,TransformerDecoder
import utils
import pickle
nltk.download('punkt')  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/wicaksonolxn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
BATCH_SIZE = 64# butuh lebih banyak update 
DATA_PATH = "dataset/"  
SAVE_DIR = "saved"

train_loader, val_loader, test_loader = get_dataloaders(
    data_path=DATA_PATH, 
    source_lang="min", 
    target_lang="eng", 
    batch_size=BATCH_SIZE, 
    device=device
)
SRC_VOCAB_SIZE = 4000
TGT_VOCAB_SIZE = 4000
N_LAYERS = 1
N_HEADS = 1
D_MODEL = 128
FFN_HIDDEN = D_MODEL*4
DROPOUT = 0.1
EPOCHS = 150

encoder = TransformerEncoder(SRC_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
decoder = TransformerDecoder(TGT_VOCAB_SIZE,D_MODEL,N_LAYERS,N_HEADS,FFN_HIDDEN,DROPOUT,device)
model = Transformer(encoder,decoder,device,utils.PAD_TOKEN).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss(ignore_index=utils.PAD_TOKEN) 
print("Model initialized on:", device)


TrainData - Max 'min' sentence length: 76
TrainData - Max 'eng' sentence length: 107
TestData - Max 'min' sentence length: 61
TestData - Max 'eng' sentence length: 75
ValidData - Max 'min' sentence length: 62
ValidData - Max 'eng' sentence length: 88
Number of examples in train_dataset,train origin,train_raw: 800 800 800
Number of examples in valid_dataset: 100
Number of examples in test_dataset: 100
Model initialized on: cuda


In [3]:
tokens = {
    "Padding": utils.PAD_TOKEN,
    "Start of Sequence": utils.SOS_TOKEN,
    "End of Sequence": utils.EOS_TOKEN,
    "Unknown": utils.UNK_TOKEN
}
for i, batch in enumerate(train_loader):
    if i < 8:
        src = batch["src"]
        tgt = batch["tgt"]
        ss,fss=src[0,:],src.shape
        st,fst=tgt[0,:],tgt.shape
        print(fss,fst)
    for name, token in tokens.items():
        print(f"{name}: {token}")
    else:
        break

torch.Size([64, 109]) torch.Size([64, 109])
Padding: 0
Start of Sequence: 1
End of Sequence: 2
Unknown: 3


testing input , is it correct

In [4]:
data_iter = iter(train_loader)
for i in range(7):
    batch = next(data_iter)
    print(batch["src"].shape)


torch.Size([64, 109])
torch.Size([64, 109])
torch.Size([64, 109])
torch.Size([64, 109])
torch.Size([64, 109])
torch.Size([64, 109])
torch.Size([64, 109])


In [5]:
for i,batch in enumerate(train_loader):
    if i <1:
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        output, _ = model(src_batch, tgt_batch[:, :-1]) 
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt_y = tgt_batch[:,1:].contiguous().view(-1)
        print(tgt_y)

tensor([ 914,  311, 2575,  ...,    0,    0,    0], device='cuda:0')


In [6]:
pth = "dataset"
src = "min"
tgt = "eng"
tp  = os.path.join(pth, f"{src}_{tgt}")
with open(os.path.join(tp, "input_dic.pkl"),  "rb") as f:
    input_dic = pickle.load(f)
with open(os.path.join(tp, "output_dic.pkl"), "rb") as f:
    output_dic = pickle.load(f)
for i, batch in enumerate(train_loader):
    if i < 1:
        src_batch = batch['src'].to(device) 
        tgt_batch = batch['tgt'].to(device) 
        batch_size = src_batch.size(0)
        num_examples = min(5, batch_size)
        for j in range(num_examples):
            src_tokens = src_batch[j].tolist()
            tgt_tokens = tgt_batch[j].tolist()
            src_sentence = utils.detokenize(src_tokens, input_dic)
            tgt_sentence = utils.detokenize(tgt_tokens, output_dic)
            print(f"Example {j+1}:")
            print("Source:", src_sentence)
            print("Target:", tgt_sentence)
            print("-" * 50)
    break  


Example 1:
Source: kini restorannyo batingkek 2. sayangnyo waktu wak ka situ acnyo mati jadi angek. dan yang mambuek wak heran, awak ndak pernah diagiah struk.
Target: now the restaurant is a twostorey. shame that it was stiflingly hot when i went there because the ac was dead. the one thing that baffled me was i was never given any receipts
--------------------------------------------------
Example 2:
Source: awak indak kecewa jo produk apple
Target: im not disappointed with apples product
--------------------------------------------------
Example 3:
Source: awak suko bana makan di siko. haragonyo indak talalu maha dan rasonyo lamak. nan paliang awak suko jamur panggangnyo. banyak kejunyo dan lembut. tiok kali ka siko pasti order iko.
Target: i really like eating here. the price is not too expensive and the flavour is wondrous. what i enjoy here the most is the grilled mushrooms. tender with lots of cheese. i order it every time i come here.
-------------------------------------------

In [7]:
import os
import torch
from tqdm import tqdm
os.makedirs(SAVE_DIR, exist_ok=True)
best_val_loss = float("inf") 
best_model_path = None
atl = {}
avl = {}
for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")
    model.train()
    total_train_loss = 0.0
    train_bar = tqdm(train_loader, desc="🚀 Training", 
                leave=True, total=len(train_loader))
    for batch in train_bar:
        optimizer.zero_grad()
        src_batch = batch['src'].to(device)
        tgt_batch = batch['tgt'].to(device)
        
        output, _ = model(src_batch, tgt_batch[:, :-1]) 
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt_y = tgt_batch[:,1:].contiguous().view(-1)
        loss = criterion(output, tgt_y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        train_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)
    atl[epoch]=avg_train_loss
    model.eval()
    total_val_loss = 0.0
    val_bar = tqdm(val_loader, desc="🚀 Validation",
              leave=True, total=len(val_loader))
    with torch.no_grad():
        for batch in val_bar:
            src_batch = batch['src'].to(device)
            tgt_batch = batch['tgt'].to(device)
            
            output, _ = model(src_batch, tgt_batch[:, :-1]) 
            output_dim = output.shape[-1]
            output = output.reshape(-1, output_dim)
            tgt_y = tgt_batch[:,1:].contiguous().view(-1)

            loss = criterion(output, tgt_y)
            total_val_loss += loss.item()
            val_bar.set_postfix(loss=f"{loss.item():.4f}")
    
    avg_val_loss = total_val_loss / len(val_loader)
    avl[epoch]=avg_val_loss
    print(f"[Epoch {epoch}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    if avg_val_loss < best_val_loss:
        if best_model_path and os.path.exists(best_model_path):
            os.remove(best_model_path)
        best_val_loss = avg_val_loss
        best_model_path = os.path.join(SAVE_DIR, "best.pt")
        torch.save(model.state_dict(), best_model_path)
        print(f"  -> New best model saved at {best_model_path}")

utils.plot_loss(atl, avl, SAVE_DIR)


Epoch 1/150


🚀 Validation: 100%|██████████| 2/2 [00:00<00:00, 98.97it/s, loss=8.7157]


[Epoch 1] Train Loss: 8.7050 | Val Loss: 8.6921
  -> New best model saved at saved/best.pt
Epoch 2/150


🚀 Validation: 100%|██████████| 2/2 [00:00<00:00, 89.89it/s, loss=8.6978]


[Epoch 2] Train Loss: 8.6838 | Val Loss: 8.6749
  -> New best model saved at saved/best.pt
Epoch 3/150


...

[Epoch 148] Train Loss: 6.7628 | Val Loss: 6.9248
  -> New best model saved at saved/best.pt
Epoch 149/150


🚀 Validation: 100%|██████████| 2/2 [00:00<00:00, 89.91it/s, loss=6.8502]


[Epoch 149] Train Loss: 6.7671 | Val Loss: 6.9228
  -> New best model saved at saved/best.pt
Epoch 150/150


🚀 Validation: 100%|██████████| 2/2 [00:00<00:00, 90.98it/s, loss=6.8479]


[Epoch 150] Train Loss: 6.7565 | Val Loss: 6.9207
  -> New best model saved at saved/best.pt
Loss plot saved to saved/loss_plot.png
